# Performance Heuristics for GR(1) Realizability Checking and Related Analyses
 Main Analysis

In [250]:
import helper_methods
import load_data
import pandas as pd
import matplotlib.pyplot as plt

import group_methods
import result_analysis
import numpy as np
import validate
from result_analysis import pivot

# Load CSV

### Mult

In [251]:
df_raw= load_data.load_data_mult()

Found 2 CSV file(s) in the 'analysis_csv' directory.
Loaded: all_runs_new_SPLIT1_ws1_20241009_235902.csv
Number of specs in this file: 80
Loaded: all_runs_new_SPLIT2_ws2_20241009_221236.csv
Number of specs in this file: 98
Total number of files loaded: 2
Total Specs: 178
Final DataFrame shape: (1068, 88)


In [252]:
df = df_raw

In [253]:
df['Spec'].isin(['166996604']).any()


False

# Result

In [254]:
baseline_runconfig='NOTHING'

In [255]:
df.columns

Index(['Spec', 'RunConfig', 'UniqueIdentifier', 'TIMEOUT', 'ActionType',
       'Result', 'CONFIG_TXT', 'INITIAL_BDD_TIME', 'HEURISTICS_TIME',
       'SECOND_STEP_TIME', 'PARALLEL_OVERHEAD_TIME', 'WORK_TIME', 'TOTAL_TIME',
       'PRE_ROYBDD_ORDER', 'POST_HEURISTICS_ORDER', 'POST_SECOND_STEP_ORDER',
       'POST_WORK_ORDER', 'PRE_ROYBDD_NODE_SIZE', 'POST_HEURISTICS_NODE_SIZE',
       'POST_SECOND_STEP_NODE_SIZE', 'POST_WORK_NODE_SIZE', 'AVERAGE_AUX_SIZE',
       'AVERAGE_SYS_DOM_IN_CONSTRAINT', 'AVERAGE_ENV_DOM_IN_CONSTRAINT',
       'AVERAGE_SYS_DOM_SIZE', 'AVERAGE_ENV_DOM_SIZE',
       'AVERAGE_CONSTRAINT_COMPLEXITY', 'INTERCONNECTNESS_INDEX',
       'TOTAL_REORDER_TIME', 'REORDER_CALL_AMOUNT', 'AVERAGE_REORDER_TIME',
       'AVERAGE_REORDER_GAIN', 'Z_FIXPOINTS', 'Y_FIXPOINTS', 'X_FIXPOINTS',
       'W_FIXPOINTS', 'REAL_CASES', 'UNREAL_CASES', 'REAL_TIME', 'UNREAL_TIME',
       'GRAPH_DATA', 'CLUSTER_INC_DATA', 'CLUSTER_VAR_DATA',
       'DOMAIN_GRAPH_DATA', 'CONSTRAINT_GRAPH_DATA', 

In [256]:
no_timeouts_df = helper_methods.get_df_rows_no_timeout(df)

In [257]:
print(f"before len: {len(df)}")

print(f"filtered len: {len(no_timeouts_df)}")

before len: 1068
filtered len: 1068


In [258]:
df_grouped_filtered = group_methods.group(no_timeouts_df)


In [259]:
print("False is expected")
# Check for NaN in the 'Result' column
print(f"any nan Result: {df_grouped_filtered['Result'].isna().any()}")

# Check for NaN in the 'WORK_TIME' column
print(f"any nan WORK_TIME: {df_grouped_filtered['WORK_TIME'].isna().any()}")

# Check for NaN in the 'TOTAL_TIME' column
print(f"any nan TOTAL_TIME: {df_grouped_filtered['TOTAL_TIME'].isna().any()}")


False is expected
any nan Result: False
any nan WORK_TIME: False
any nan TOTAL_TIME: False


In [260]:
helper_methods.explore_amount_of_configs(df_grouped_filtered)


Unique counts of RunConfigs per Spec: [2]


### Different Results

In [261]:
def first_if_eq(s):
    if s.nunique() == 1:
        return s.iloc[0]
    raise ValueError(f"Not all values are equal for Spec: {s.name}, {s}")

result = (helper_methods.get_df_specs_no_timeout(df_grouped_filtered)
          .groupby('Spec')['Result']
          .agg(first_if_eq)
          .value_counts())

print(result)

Result
STRONG        123
NOT_STRONG     55
Name: count, dtype: int64


In [262]:
def first_if_eq(s):
    if s.nunique() == 1:
        return s.iloc[0]
    raise ValueError(f"Not all values are equal for Spec: {s.name}, {s}")

def process_data(df_grouped_filtered):
    specs_df = helper_methods.get_df_specs_no_timeout(df_grouped_filtered)
    
    equal_value_groups = []
    unequal_value_groups = []
    
    for spec, group in specs_df.groupby('Spec'):
        try:
            first_if_eq(group['Result'])
            equal_value_groups.append(spec)
        except ValueError:
            unequal_value_groups.append(spec)
    
    result = (pd.Series(equal_value_groups)
              .value_counts())
    
    print("Groups with equal values:")
    print(result)
    
    print("\nGroups without equal values:")
    for spec in unequal_value_groups:
        print(f"- {spec}")

# Assuming df_grouped_filtered is already defined
process_data(df_grouped_filtered)

Groups with equal values:
1669468393     1
1672504085     1
1672060474     1
1672060600     1
1672060813     1
              ..
1670525188     1
1670538229     1
1670541413     1
1670542951     1
variant-3-9    1
Name: count, Length: 178, dtype: int64

Groups without equal values:


## Basic Results - WORKTIME

### Percent of TIMEOUT Calc 1

In [263]:
def count_timeouts_per_runconfig(df):
    # Group by RunConfig and count Specs with TIMEOUT==1
    timeout_counts = df[df['TIMEOUT'] == 1].groupby('RunConfig')['Spec'].nunique()
    
    # Get total number of unique Specs for each RunConfig
    total_specs = df.groupby('RunConfig')['Spec'].nunique()
    
    # Combine counts and calculate percentages
    result = pd.DataFrame({
        'Timeout_Count': timeout_counts,
        'Total_Specs': total_specs,
        'Timeout_Percentage': (timeout_counts / total_specs * 100).round(2)
    }).sort_values('Timeout_Count', ascending=False)
    
    # Print results
    for runconfig, row in result.iterrows():
        print(f"{runconfig}: {row['Timeout_Count']} / {row['Total_Specs']} Specs "
              f"({row['Timeout_Percentage']}%) timed out")

# Assuming your DataFrame is named 'df'
count_timeouts_per_runconfig(df)


FINAL: nan / 178.0 Specs (nan%) timed out
NOTHING: nan / 178.0 Specs (nan%) timed out


In [264]:
def count_TIMEOUT(series):
    return (series == 1).sum()

def count_timeouts_per_runconfig1(df):
    # Group by 'RunConfig' and aggregate counts
    result1 = df.groupby('RunConfig').agg(
        Timeout_Count=('TIMEOUT', count_TIMEOUT),
        Total_Specs=('Spec', 'nunique')
    )
    
    # Calculate timeout percentage
    result1['Timeout_Percentage'] = (result1['Timeout_Count'] / result1['Total_Specs'] * 100).round(2)
    
    # Sort by 'Timeout_Count' in descending order
    result = result1.sort_values('Timeout_Count', ascending=False)
    
    # Print results
    for runconfig, row in result.iterrows():
        print(f"{runconfig}: {int(row['Timeout_Count'])} / {row['Total_Specs']} Specs "
              f"({row['Timeout_Percentage']}%) timed out")
        
count_timeouts_per_runconfig1(df)

result1 = df.groupby('RunConfig').agg(
    Timeout_Count=('TIMEOUT', count_TIMEOUT),
    Total_Specs=('Spec', 'nunique')
)
result1

FINAL: 0 / 178.0 Specs (0.0%) timed out
NOTHING: 0 / 178.0 Specs (0.0%) timed out


,Timeout_Count,Total_Specs
RunConfig,,
FINAL,0,178
NOTHING,0,178


### Amount reduced

In [265]:
pivot_df_reduced = result_analysis.pivot(df_grouped_filtered, ['WORK_TIME', 'TOTAL_TIME'])

# Get all columns except the baseline
columns = [col for col in pivot_df_reduced['TOTAL_TIME'].columns if col != baseline_runconfig]

# Create head-to-head comparisons
headtoheads_columns = [(col,[baseline_runconfig, col]) for col in columns]
print(f"head to head values: {headtoheads_columns}")

head to head values: [('FINAL', ['NOTHING', 'FINAL'])]


In [266]:
def calc_smaller(pivot_df,headtoheads_columns, baseline_runconfig):
    pivot_df = pivot_df.copy()
    
    # Head-to-head comparisons
    for name, cols in headtoheads_columns:
        # Extract the relevant columns for this comparison
        pivot_df_headtohead = pivot_df['TOTAL_TIME'][cols]
    
        # Filter rows where both are NaN
        pivot_df_headtohead = pivot_df_headtohead.dropna(how='all')
        # If only one is TIMEOUT, let the other 'win'
        pivot_df_headtohead = pivot_df_headtohead.fillna(9999999)
        nonbaseline = cols[1]
    
        row_smaller_amount = (pivot_df_headtohead[nonbaseline] < pivot_df_headtohead[baseline_runconfig]).sum()
    
    
    
        # Calculate percentage smaller than baseline
        percentage =  row_smaller_amount / len(pivot_df_headtohead) * 100
    
        print(f"{cols[1]}: {percentage:.2f}% smaller than baseline")
        print(f"{cols[1]}: {row_smaller_amount:.2f} {len(pivot_df_headtohead)}")


print("ALL")
calc_smaller(pivot_df_reduced,headtoheads_columns, baseline_runconfig)
print()
print("BIGGER THAN 60 secs")
calc_smaller(pivot_df_reduced[pivot_df_reduced['WORK_TIME']['NOTHING'] > 60000],headtoheads_columns, baseline_runconfig)


ALL
FINAL: 82.02% smaller than baseline
FINAL: 146.00 178

BIGGER THAN 60 secs
FINAL: 100.00% smaller than baseline
FINAL: 3.00 3


In [267]:
pivot_df_reduced

TOTAL_TIME                    WORK_TIME               
RunConfig            FINAL        NOTHING         FINAL        NOTHING
Spec                                                                  
1669468393     2400.666667    2328.000000    405.666667     364.666667
1669640935     2578.666667    3223.666667    500.333333     921.666667
1669643093     2426.333333    3583.666667    281.000000    1193.000000
1669643605     2396.333333    3510.666667    240.000000    1166.666667
1669665995   128850.333333  574895.333333  90256.666667  495372.000000
...                    ...            ...           ...            ...
variant-2-4    2870.000000    3416.333333    828.000000    1296.666667
variant-2-8    1969.000000    2163.000000     16.000000      93.666667
variant-3-1    5307.333333    3692.000000   3146.000000     875.000000
variant-3-4    2385.333333    2307.333333     31.333333      31.000000
variant-3-9    3750.000000    3740.000000   1463.666667     666.666667

[178 rows x 4 columns]

### Calculate

## Ratio

In [268]:
df_grouped = group_methods.group(df)
print(f"len {len(df_grouped)}")

len 356


In [269]:
# Used in RATIO: Keep only specs that have no timeout
df_grouped_no_timeout = helper_methods.get_df_specs_no_timeout(df_grouped)

# Not used in RATIO: Keep only specs that have no timeout & doesn't change the result
df_grouped_no_timeout_same_result = helper_methods.get_df_specs_same_column(df_grouped_no_timeout, 'Result')

In [270]:
print(f"amount of no timeout runs {len(df_grouped_no_timeout)}")
print(f"amount of no timeout runs & same result {len(df_grouped_no_timeout_same_result)}")

print(f"amount of no timeout specs {len(df_grouped_no_timeout['Spec'].unique())}")
print(f"amount of no timeout specs & same result {len(df_grouped_no_timeout_same_result['Spec'].unique())}")

print(f"Any missing no_timeout: {df_grouped_no_timeout['WORK_TIME'].isna().any()}")
print(f"Any missing no_timeout_same_result: {df_grouped_no_timeout_same_result['WORK_TIME'].isna().any()}")

amount of no timeout runs 356
amount of no timeout runs & same result 356
amount of no timeout specs 178
amount of no timeout specs & same result 178
Any missing no_timeout: False
Any missing no_timeout_same_result: False


In [271]:
metric_columns = ['WORK_TIME','TOTAL_TIME','TOTAL_REORDER_TIME']
pivot_df = result_analysis.pivot(df_grouped_no_timeout,metric_columns )

runconfigs = df_grouped_no_timeout['RunConfig'].unique()
pivot_df

TOTAL_REORDER_TIME                    TOTAL_TIME                 \
RunConfig                FINAL        NOTHING          FINAL        NOTHING   
Spec                                                                          
1669468393          350.666667     289.333333    2400.666667    2328.000000   
1669640935          511.666667     931.666667    2578.666667    3223.666667   
1669643093          379.333333    1118.666667    2426.333333    3583.666667   
1669643605          371.333333    1007.333333    2396.333333    3510.666667   
1669665995       124155.666667  561568.666667  128850.333333  574895.333333   
...                        ...            ...            ...            ...   
variant-2-4         852.000000    1431.666667    2870.000000    3416.333333   
variant-2-8          34.000000     217.000000    1969.000000    2163.000000   
variant-3-1        2467.666667    1438.666667    5307.333333    3692.000000   
variant-3-4         235.000000     172.000000    2385.333333    2307.333333   
variant-3-9        1336.666667    1317.666667    3750.000000    3740.000000   

                WORK_TIME                 
RunConfig           FINAL        NOTHING  
Spec                                      
1669468393     405.666667     364.666667  
1669640935     500.333333     921.666667  
1669643093     281.000000    1193.000000  
1669643605     240.000000    1166.666667  
1669665995   90256.666667  495372.000000  
...                   ...            ...  
variant-2-4    828.000000    1296.666667  
variant-2-8     16.000000      93.666667  
variant-3-1   3146.000000     875.000000  
variant-3-4     31.333333      31.000000  
variant-3-9   1463.666667     666.666667  

[178 rows x 6 columns]

In [272]:
print("MODE:all")
print(f'SPEC AMOUNT: {len(pivot_df)}')
geometric_averages = result_analysis.get_geometric_avg(pivot_df,runconfigs,baseline_runconfig)


MODE:all
SPEC AMOUNT: 178
is_any_null_or_nonpositive True
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive True
is_any_null_or_nonpositive False
                               Geometric Average
Heuristic                                       
(TOTAL_REORDER_TIME, FINAL)             0.000000
(TOTAL_REORDER_TIME, NOTHING)           1.000000
(TOTAL_TIME, FINAL)                     0.830938
(TOTAL_TIME, NOTHING)                   1.000000
(WORK_TIME, FINAL)                      0.000000
(WORK_TIME, NOTHING)                    1.000000


In [273]:
def above_threshold_geo_avg(pivot_df, runconfigs, baseline_runconfig, percentile=80):
    print(f"MODE: top {100-percentile}%")
        
    # Calculate the Xth percentile value
    threshold = pivot_df['WORK_TIME']['NOTHING'].quantile(percentile/100)
    
    # Keep only the rows above or equal to the threshold
    pivot_df_temp = pivot_df[pivot_df['WORK_TIME']['NOTHING'] >= threshold]
    
    print(f'len {len(pivot_df_temp)}')
    
    geometric_averages = result_analysis.get_geometric_avg(pivot_df_temp, runconfigs, baseline_runconfig)
    
    return geometric_averages


### Ratio based on top %

In [274]:
above_threshold_geo_avg(pivot_df,runconfigs,baseline_runconfig,percentile=50)

MODE: top 50%
len 89
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
                               Geometric Average
Heuristic                                       
(TOTAL_REORDER_TIME, FINAL)             0.537182
(TOTAL_REORDER_TIME, NOTHING)           1.000000
(TOTAL_TIME, FINAL)                     0.836443
(TOTAL_TIME, NOTHING)                   1.000000
(WORK_TIME, FINAL)                      0.634430
(WORK_TIME, NOTHING)                    1.000000


{('TOTAL_REORDER_TIME', 'FINAL'): 0.5371817634528933,
 ('TOTAL_REORDER_TIME', 'NOTHING'): 1.0,
 ('TOTAL_TIME', 'FINAL'): 0.8364434831660321,
 ('TOTAL_TIME', 'NOTHING'): 1.0,
 ('WORK_TIME', 'FINAL'): 0.6344303856387596,
 ('WORK_TIME', 'NOTHING'): 1.0}

In [275]:
above_threshold_geo_avg(pivot_df,runconfigs,baseline_runconfig,percentile=80)

MODE: top 20%
len 36
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
                               Geometric Average
Heuristic                                       
(TOTAL_REORDER_TIME, FINAL)             0.551000
(TOTAL_REORDER_TIME, NOTHING)           1.000000
(TOTAL_TIME, FINAL)                     0.741491
(TOTAL_TIME, NOTHING)                   1.000000
(WORK_TIME, FINAL)                      0.576674
(WORK_TIME, NOTHING)                    1.000000


{('TOTAL_REORDER_TIME', 'FINAL'): 0.5510003002170973,
 ('TOTAL_REORDER_TIME', 'NOTHING'): 1.0,
 ('TOTAL_TIME', 'FINAL'): 0.7414907599688618,
 ('TOTAL_TIME', 'NOTHING'): 1.0,
 ('WORK_TIME', 'FINAL'): 0.5766741627433969,
 ('WORK_TIME', 'NOTHING'): 1.0}

In [276]:
# Ratio
above_threshold_geo_avg(pivot_df,runconfigs,baseline_runconfig,percentile=50)

MODE: top 50%
len 89
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
is_any_null_or_nonpositive False
                               Geometric Average
Heuristic                                       
(TOTAL_REORDER_TIME, FINAL)             0.537182
(TOTAL_REORDER_TIME, NOTHING)           1.000000
(TOTAL_TIME, FINAL)                     0.836443
(TOTAL_TIME, NOTHING)                   1.000000
(WORK_TIME, FINAL)                      0.634430
(WORK_TIME, NOTHING)                    1.000000


{('TOTAL_REORDER_TIME', 'FINAL'): 0.5371817634528933,
 ('TOTAL_REORDER_TIME', 'NOTHING'): 1.0,
 ('TOTAL_TIME', 'FINAL'): 0.8364434831660321,
 ('TOTAL_TIME', 'NOTHING'): 1.0,
 ('WORK_TIME', 'FINAL'): 0.6344303856387596,
 ('WORK_TIME', 'NOTHING'): 1.0}

In [277]:
metric_columns = ['WORK_TIME','TOTAL_TIME','TOTAL_REORDER_TIME','Y_FIXPOINTS']
pivot_df = result_analysis.pivot(df_grouped_no_timeout,metric_columns )

unreal = df.groupby('Spec')['Result'].agg(lambda x: all(x=='SYS_UNREAL'))
pivot_df_unreal = pivot_df[pivot_df.index.isin(unreal[unreal].index)]


print("MODE:unreal")
print(f'SPEC AMOUNT: {len(pivot_df)}')
geometric_averages = result_analysis.get_geometric_avg(pivot_df_unreal,runconfigs,baseline_runconfig)

MODE:unreal
SPEC AMOUNT: 178


KeyError: 'Y_FIXPOINTS'